In [23]:
import derive
reload(derive)

fun = lambda x: (3 * x[0] + x[1]) ** 2

print derive.hess(fun)([3, 1])

[[ 18.00160021   6.0005334 ]
 [  6.0005334    2.00373051]]


In [49]:
import sympy as sm

x = sm.symbols(' '.join(['x%s' % i for i in xrange(3)]))
fun = (3 * x[0] + x[1]) ** 2
g = sm.diff(fun, x[0])
print g.subs(x[0], 3)
f = sm.lambdify(x[0], g, 'numpy')
print f(3)

def partial(fun, i, x):
    v = x[:]
    def wraps(x):
        v[i] = x
        return fun(v)
    return sm.diff(wraps, x[i])

6*x1 + 54
6*x1 + 54


In [25]:
import derive
reload(derive)

import tensorflow as tf      

x = tf.Variable(0, dtype = tf.float32)
y = tf.Variable(0, dtype = tf.float32)
f = (3 * x + y) ** 2
hess = derive.hess(f)

with tf.Session() as sess:
    sess.run(tf.initialize_all_variables())
    print hess([x, y]).eval()

[[ 18.   6.]
 [  6.   2.]]


In [211]:
import theano
import theano.tensor as T

x = T.dvector('x')
y = x ** 2
cost = y.sum()
gy = T.grad(cost, x)
H, updates = theano.scan(lambda i, gy,x : T.grad(gy[i], x), sequences=T.arange(gy.shape[0]), non_sequences=[gy, x])
f = theano.function([x], H, updates=updates)